### IMPORT LIBRARY

In [ ]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize
import time
import openpyxl
from openpyxl.styles import Alignment, PatternFill
from gensim.models import Word2Vec
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

# TF-IDF

In [6]:
def preprocess_text_simple(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    return text.strip()

def load_and_preprocess_course_data(file_path):
    df = pd.read_csv(file_path)
    df.drop(columns=['Unnamed: 0','Program Type', 'Courses', 'Level', 'Number of Reviews',
           'Unique Projects', 'Prequisites', 'What you learn', 'Related Programs',
           'Monthly access', '6-Month access', '4-Month access', '3-Month access',
           '5-Month access', '2-Month access', 'School', 'Topics related to CRM',
           'ExpertTracks', 'FAQs', 'Course Title', 'Course URL',
           'Course Short Intro', 'Weekly study', 'Premium course',
           "What's include", 'Rank', 'Created by', 'Program', 'Number of ratings',
           'Price', 'COURSE CATEGORIES'], inplace=True)
    
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.drop_duplicates(subset=['Title', 'Short Intro'])
    
    translations = {
        '计算机科学': 'Computer Science',
        'Ciencia de Datos': 'Data Science',
        'Negocios': 'Business',
        'Ciencias de la Computación': 'Computer Science',
        'Negócios': 'Business',
        'データサイエンス': 'Data Science',
        'Tecnologia da informação': 'Information Technology'
    }
    df['Category'] = df['Category'].replace(translations)
    
    df['Rating'] = df['Rating'].str.replace('stars', '', regex=False)
    df['Number of viewers'] = df['Number of viewers'].str.replace(r'\D+', '', regex=True)
    
    df['combined'] = df['Title'] + ' ' + df['Short Intro'].fillna('') + ' ' + df['Skills'].fillna('') + ' ' +df['Category'].fillna('') + ' ' + df['Sub-Category'].fillna('')
    df['combined'] = df['combined'].apply(preprocess_text_simple)
    
    keywords = ['Participant', 'Designed', 'Learners', 'prior', 'experience']
    df['Subtitle Languages'] = df['Subtitle Languages'].apply(lambda x: np.nan if any(keyword in str(x) for keyword in keywords) else x)
    
    fill_columns = ['Instructors', 'Duration', 'Site', 'Course Type', 'Language', 'Subtitle Languages', 'Category', 'Sub-Category', 'Short Intro', 'Skills']
    for col in fill_columns:
        df[col] = df[col].fillna('Unknown')
    
    df['Number of viewers'] = pd.to_numeric(df['Number of viewers'], errors='coerce').fillna(0)
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0)
    
    return df

def recommend(user_input, df, vectorizer, tfidf_matrix):
    user_input_processed = preprocess_text_simple(user_input)
    user_tfidf = vectorizer.transform([user_input_processed])
    
    cosine_similarities = cosine_similarity(user_tfidf, tfidf_matrix).flatten()
    
    top_course_indices = cosine_similarities.argsort()[::-1]
    
    recommendations = df.iloc[top_course_indices].copy()
    recommendations['cosine_similarity'] = cosine_similarities[top_course_indices]
    
    return recommendations

def imdb_score(df, q=0.95):
    df = df.copy()
    m = df['Number of viewers'].quantile(q)
    c = (df['Rating'] * df['Number of viewers']).sum() / df['Number of viewers'].sum()
    df["score"] = df.apply(lambda x: (x.Rating * x['Number of viewers'] + c*m) / (x['Number of viewers'] + m), axis=1)
    return df  

def process_test_cases_and_save(test_cases_file, course_data_file, output_file):
    df = load_and_preprocess_course_data(course_data_file)
    
    vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df['combined'])
    
    test_cases_df = pd.read_csv(test_cases_file)
    
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Course Recommendations"
    
    headers = ['No', 'User Preferences', 'Recommended Courses', 'Scores']
    for col, header in enumerate(headers, start=1):
        ws.cell(row=1, column=col, value=header)
    
    row_counter = 2
    for idx, row in test_cases_df.iterrows():
        user_input = row['Case']
        
        recommendations = recommend(user_input, df, vectorizer, tfidf_matrix)
        
        percentile_threshold = 95
        threshold_value = np.percentile(recommendations['cosine_similarity'], percentile_threshold)
        stage1 = recommendations[recommendations['cosine_similarity'] >= threshold_value]
        
        stage2 = imdb_score(stage1)
        
        stage2['score'] = (stage2['score'] - stage2['score'].min()) / (stage2['score'].max() - stage2['score'].min())
        stage2['cosine_similarity'] = (stage2['cosine_similarity'] - stage2['cosine_similarity'].min()) / (stage2['cosine_similarity'].max() - stage2['cosine_similarity'].min())
        
        stage2['Final'] = 0.5 * stage2['cosine_similarity'] + 0.5 * stage2['score']
        stage2 = stage2.sort_values(by='Final', ascending=False)
        
        threshold_value = np.percentile(stage2['Final'], percentile_threshold)
        recommendations_final = stage2[stage2['Final'] >= threshold_value]
        
        rec_titles = recommendations_final['Title'].tolist()[:10]
        scores = recommendations_final['Final'].round(4).tolist()[:10]
        
        ws.cell(row=row_counter, column=1, value=idx + 1)
        ws.cell(row=row_counter, column=2, value=user_input)
        
        for rec, score in zip(rec_titles, scores):
            ws.cell(row=row_counter, column=3, value=rec)
            ws.cell(row=row_counter, column=4, value=score)
            row_counter += 1
        
        for _ in range(10 - len(rec_titles)):
            ws.cell(row=row_counter, column=3, value="")
            ws.cell(row=row_counter, column=4, value="")
            row_counter += 1
    
    for row in range(2, ws.max_row, 10):
        ws.merge_cells(start_row=row, start_column=1, end_row=row+9, end_column=1)
        ws.merge_cells(start_row=row, start_column=2, end_row=row+9, end_column=2)
    
    header_fill = PatternFill(start_color="CCCCCC", end_color="CCCCCC", fill_type="solid")
    for cell in ws[1]:
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        for cell in row:
            cell.alignment = Alignment(vertical='top', wrap_text=True)
    
    ws.column_dimensions['A'].width = 5
    ws.column_dimensions['B'].width = 50
    ws.column_dimensions['C'].width = 50
    ws.column_dimensions['D'].width = 15
    
    wb.save(output_file)
    print(f"Course recommendations saved to '{output_file}'")

if __name__ == "__main__":
    test_cases_file = '/kaggle/input/benerdong/Updated_User_Preferences_Comma_Delimited.csv'
    course_data_file = '/kaggle/input/online-courses/Online_Courses.csv'
    output_file = 'course_recommendations_tfidf.xlsx'
    
    process_test_cases_and_save(test_cases_file, course_data_file, output_file)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/tmp/ipykernel_34/674011380.py:36: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


Recommendation time for input 'I want to gain hands-on experience in developing, deploying, and optimizing machine learning models. My goal is to enhance my skills in data preprocessing, feature engineering, model selection, and hyperparameter tuning. Additionally, I seek to understand the end-to-end machine learning pipeline, including model deployment in production environments and monitoring model performance in real-world applications': 0.01 seconds
Recommendation time for input 'I want to gain the knowledge and skills necessary to launch and grow a successful business. My goal is to understand the fundamentals of business planning, market research, and financial management. Additionally, I aim to learn about effective marketing strategies, customer acquisition, and the legal aspects of starting a business to ensure long-term success.': 0.00 seconds
Recommendation time for input 'I am interested in acquiring skills in data analysis, statistical modeling, and data visualization to e

# WORD2VEC

In [ ]:
def preprocess_text_simple(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    return text.strip()

def load_and_preprocess_course_data(file_path):
    df = pd.read_csv(file_path)
    df.drop(columns=['Unnamed: 0','Program Type', 'Courses', 'Level', 'Number of Reviews',
           'Unique Projects', 'Prequisites', 'What you learn', 'Related Programs',
           'Monthly access', '6-Month access', '4-Month access', '3-Month access',
           '5-Month access', '2-Month access', 'School', 'Topics related to CRM',
           'ExpertTracks', 'FAQs', 'Course Title', 'Course URL',
           'Course Short Intro', 'Weekly study', 'Premium course',
           "What's include", 'Rank', 'Created by', 'Program', 'Number of ratings',
           'Price', 'COURSE CATEGORIES'], inplace=True)
    
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.drop_duplicates(subset=['Title', 'Short Intro'])
    
    translations = {
        '计算机科学': 'Computer Science',
        'Ciencia de Datos': 'Data Science',
        'Negocios': 'Business',
        'Ciencias de la Computación': 'Computer Science',
        'Negócios': 'Business',
        'データサイエンス': 'Data Science',
        'Tecnologia da informação': 'Information Technology'
    }
    df['Category'] = df['Category'].replace(translations)
    
    df['Rating'] = df['Rating'].str.replace('stars', '', regex=False)
    df['Number of viewers'] = df['Number of viewers'].str.replace(r'\D+', '', regex=True)
    
    df['combined'] = df['Title'] + ' ' + df['Short Intro'].fillna('') + ' ' + df['Skills'].fillna('') + ' ' + df['Category'].fillna('') + ' ' + df['Sub-Category'].fillna('')
    df['combined'] = df['combined'].apply(preprocess_text_simple)
    
    keywords = ['Participant', 'Designed', 'Learners', 'prior', 'experience']
    df['Subtitle Languages'] = df['Subtitle Languages'].apply(lambda x: np.nan if any(keyword in str(x) for keyword in keywords) else x)
    
    fill_columns = ['Instructors', 'Duration', 'Site', 'Course Type', 'Language', 'Subtitle Languages', 'Category', 'Sub-Category', 'Short Intro', 'Skills']
    for col in fill_columns:
        df[col] = df[col].fillna('Unknown')
    
    df['Number of viewers'] = pd.to_numeric(df['Number of viewers'], errors='coerce').fillna(0).astype(int)
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0)
    
    df['Tokenized'] = df['combined'].apply(word_tokenize)
    
    return df

def train_word2vec(df):
    model = Word2Vec(sentences=df['Tokenized'], vector_size=100, window=5, min_count=1, workers=4)
    return model

def get_document_vector(doc, model):
    words = word_tokenize(doc)
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) == 0:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

def vectorize_text(df, model):
    doc_vectors = np.array([get_document_vector(doc, model) for doc in df['combined']])
    return doc_vectors

def imdb_score(df, q=0.95):
    df = df.copy()
    m = df['Number of viewers'].quantile(q)
    c = (df['Rating'] * df['Number of viewers']).sum() / df['Number of viewers'].sum()
    df["score"] = df.apply(lambda x: (x.Rating * x['Number of viewers'] + c*m) / (x['Number of viewers'] + m), axis=1)
    return df

def recommend_course(user_input, df, model, doc_vectors):
    user_input_processed = preprocess_text_simple(user_input)
    user_vector = get_document_vector(user_input_processed, model)
    
    cosine_similarities = cosine_similarity([user_vector], doc_vectors).flatten()
    
    df_temp = df.copy()
    df_temp['cosine_similarity'] = cosine_similarities
    
    percentile_threshold = 95
    threshold_value = np.percentile(df_temp['cosine_similarity'], percentile_threshold)
    stage1 = df_temp[df_temp['cosine_similarity'] >= threshold_value]
    
    stage2 = imdb_score(stage1)
    stage2['score'] = (stage2['score'] - stage2['score'].min()) / (stage2['score'].max() - stage2['score'].min())
    stage2['cosine_similarity'] = (stage2['cosine_similarity'] - stage2['cosine_similarity'].min()) / (stage2['cosine_similarity'].max() - stage2['cosine_similarity'].min())
    
    stage2['Final'] = 0.5 * stage2['cosine_similarity'] + 0.5 * stage2['score']
    stage2 = stage2.sort_values(by='Final', ascending=False)
    
    threshold_value = np.percentile(stage2['Final'], percentile_threshold)
    recommendations_final = stage2[stage2['Final'] >= threshold_value]
    
    return recommendations_final

def process_test_cases_and_save(test_cases_file, course_data_file, output_file, model, doc_vectors):
    test_cases_df = pd.read_csv(test_cases_file)
    df = load_and_preprocess_course_data(course_data_file)

    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Course Recommendations"

    headers = ['No', 'User Preferences', 'Recommended Courses', 'Scores']
    for col, header in enumerate(headers, start=1):
        ws.cell(row=1, column=col, value=header)

    row_counter = 2
    for idx, row in test_cases_df.iterrows():
        user_input = row['Case']
        
        recommendations = recommend_course(user_input, df, model, doc_vectors)
        
        if not recommendations.empty:
            rec_titles = recommendations['Title'].tolist()[:10]
            scores = recommendations['Final'].round(4).tolist()[:10]
        else:
            rec_titles = ["No relevant courses found"]
            scores = ["N/A"]

        ws.cell(row=row_counter, column=1, value=idx + 1)
        ws.cell(row=row_counter, column=2, value=user_input)
        
        for rec, score in zip(rec_titles, scores):
            ws.cell(row=row_counter, column=3, value=rec)
            ws.cell(row=row_counter, column=4, value=score)
            row_counter += 1

        for _ in range(10 - len(rec_titles)):
            ws.cell(row=row_counter, column=3, value="")
            ws.cell(row=row_counter, column=4, value="")
            row_counter += 1

    for row in range(2, ws.max_row, 10):
        ws.merge_cells(start_row=row, start_column=1, end_row=row+9, end_column=1)
        ws.merge_cells(start_row=row, start_column=2, end_row=row+9, end_column=2)

    header_fill = PatternFill(start_color="CCCCCC", end_color="CCCCCC", fill_type="solid")
    for cell in ws[1]:
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        for cell in row:
            cell.alignment = Alignment(vertical='top', wrap_text=True)

    ws.column_dimensions['A'].width = 5
    ws.column_dimensions['B'].width = 50
    ws.column_dimensions['C'].width = 50
    ws.column_dimensions['D'].width = 15

    wb.save(output_file)
    print(f"Course recommendations saved to '{output_file}'")

if __name__ == "__main__":
    test_cases_file = '/kaggle/input/benerdong/Updated_User_Preferences_Comma_Delimited.csv'
    course_data_file = '/kaggle/input/online-courses/Online_Courses.csv'
    output_file = 'course_recommendations_word2vec.xlsx'
    
    df = load_and_preprocess_course_data(course_data_file)
    word2vec_model = train_word2vec(df)
    doc_vectors = vectorize_text(df, word2vec_model)
    
    process_test_cases_and_save(test_cases_file, course_data_file, output_file, word2vec_model, doc_vectors)

# BERT

In [1]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def preprocess_text_simple(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    return text.strip()

def load_and_preprocess_course_data(file_path):
    df = pd.read_csv(file_path)
    df.drop(columns=['Unnamed: 0','Program Type', 'Courses', 'Level', 'Number of Reviews',
           'Unique Projects', 'Prequisites', 'What you learn', 'Related Programs',
           'Monthly access', '6-Month access', '4-Month access', '3-Month access',
           '5-Month access', '2-Month access', 'School', 'Topics related to CRM',
           'ExpertTracks', 'FAQs', 'Course Title', 'Course URL',
           'Course Short Intro', 'Weekly study', 'Premium course',
           "What's include", 'Rank', 'Created by', 'Program', 'Number of ratings',
           'Price', 'COURSE CATEGORIES'], inplace=True)
    
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    df = df.drop_duplicates(subset=['Title', 'Short Intro'])
    
    translations = {
        '计算机科学': 'Computer Science',
        'Ciencia de Datos': 'Data Science',
        'Negocios': 'Business',
        'Ciencias de la Computación': 'Computer Science',
        'Negócios': 'Business',
        'データサイエンス': 'Data Science',
        'Tecnologia da informação': 'Information Technology'
    }
    df['Category'] = df['Category'].replace(translations)
    
    df['Rating'] = df['Rating'].str.replace('stars', '', regex=False)
    df['Number of viewers'] = df['Number of viewers'].str.replace(r'\D+', '', regex=True)
    
    df['combined'] = df['Title'] + ' ' + df['Short Intro'].fillna('') + ' ' + df['Skills'].fillna('') + ' ' + df['Category'].fillna('') + ' ' + df['Sub-Category'].fillna('')
    df['combined'] = df['combined'].apply(preprocess_text_simple)
    
    keywords = ['Participant', 'Designed', 'Learners', 'prior', 'experience']
    df['Subtitle Languages'] = df['Subtitle Languages'].apply(lambda x: np.nan if any(keyword in str(x) for keyword in keywords) else x)
    
    fill_columns = ['Instructors', 'Duration', 'Site', 'Course Type', 'Language', 'Subtitle Languages', 'Category', 'Sub-Category', 'Short Intro', 'Skills']
    for col in fill_columns:
        df[col] = df[col].fillna('Unknown')
    
    df['Number of viewers'] = pd.to_numeric(df['Number of viewers'], errors='coerce').fillna(0).astype(int)
    df['Rating'] = pd.to_numeric(df['Rating'], errors='coerce').fillna(0)
    
    return df

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

def vectorize_text(df):
    doc_vectors = []
    for doc in tqdm(df['combined'], desc="Vectorizing documents"):
        doc_vectors.append(get_bert_embedding(doc))
    return np.array(doc_vectors)

def imdb_score(df, q=0.95):
    df = df.copy()
    m = df['Number of viewers'].quantile(q)
    c = (df['Rating'] * df['Number of viewers']).sum() / df['Number of viewers'].sum()
    df["score"] = df.apply(lambda x: (x.Rating * x['Number of viewers'] + c*m) / (x['Number of viewers'] + m), axis=1)
    return df

def recommend_course(user_input, df, doc_vectors):
    user_input_processed = preprocess_text_simple(user_input)
    user_vector = get_bert_embedding(user_input_processed)
    
    cosine_similarities = cosine_similarity([user_vector], doc_vectors).flatten()
    
    df_temp = df.copy()
    df_temp['cosine_similarity'] = cosine_similarities
    
    percentile_threshold = 95
    threshold_value = np.percentile(df_temp['cosine_similarity'], percentile_threshold)
    stage1 = df_temp[df_temp['cosine_similarity'] >= threshold_value]
    
    stage2 = imdb_score(stage1)
    stage2['score'] = (stage2['score'] - stage2['score'].min()) / (stage2['score'].max() - stage2['score'].min())
    stage2['cosine_similarity'] = (stage2['cosine_similarity'] - stage2['cosine_similarity'].min()) / (stage2['cosine_similarity'].max() - stage2['cosine_similarity'].min())
    
    stage2['Final'] = 0.5 * stage2['cosine_similarity'] + 0.5 * stage2['score']
    stage2 = stage2.sort_values(by='Final', ascending=False)
    
    threshold_value = np.percentile(stage2['Final'], percentile_threshold)
    recommendations_final = stage2[stage2['Final'] >= threshold_value]
    
    return recommendations_final

def process_test_cases_and_save(test_cases_file, course_data_file, output_file, doc_vectors):
    test_cases_df = pd.read_csv(test_cases_file)
    df = load_and_preprocess_course_data(course_data_file)

    wb = openpyxl.Workbook()
    ws = wb.active
    ws.title = "Course Recommendations"

    headers = ['No', 'User Preferences', 'Recommended Courses', 'Scores']
    for col, header in enumerate(headers, start=1):
        ws.cell(row=1, column=col, value=header)

    row_counter = 2
    for idx, row in tqdm(test_cases_df.iterrows(), total=len(test_cases_df), desc="Processing test cases"):
        user_input = row['Case']
        
        recommendations = recommend_course(user_input, df, doc_vectors)
        
        if not recommendations.empty:
            rec_titles = recommendations['Title'].tolist()[:10]
            scores = recommendations['Final'].round(4).tolist()[:10]
        else:
            rec_titles = ["No relevant courses found"]
            scores = ["N/A"]

        ws.cell(row=row_counter, column=1, value=idx + 1)
        ws.cell(row=row_counter, column=2, value=user_input)
        
        for rec, score in zip(rec_titles, scores):
            ws.cell(row=row_counter, column=3, value=rec)
            ws.cell(row=row_counter, column=4, value=score)
            row_counter += 1

        for _ in range(10 - len(rec_titles)):
            ws.cell(row=row_counter, column=3, value="")
            ws.cell(row=row_counter, column=4, value="")
            row_counter += 1

    for row in range(2, ws.max_row, 10):
        ws.merge_cells(start_row=row, start_column=1, end_row=row+9, end_column=1)
        ws.merge_cells(start_row=row, start_column=2, end_row=row+9, end_column=2)

    header_fill = PatternFill(start_color="CCCCCC", end_color="CCCCCC", fill_type="solid")
    for cell in ws[1]:
        cell.fill = header_fill
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)

    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        for cell in row:
            cell.alignment = Alignment(vertical='top', wrap_text=True)

    ws.column_dimensions['A'].width = 5
    ws.column_dimensions['B'].width = 50
    ws.column_dimensions['C'].width = 50
    ws.column_dimensions['D'].width = 15

    wb.save(output_file)
    print(f"Course recommendations saved to '{output_file}'")

if __name__ == "__main__":
    test_cases_file = '/kaggle/input/benerdong/Updated_User_Preferences_Comma_Delimited.csv'
    course_data_file = '/kaggle/input/online-courses/Online_Courses.csv'
    output_file = 'course_recommendations_bert.xlsx'
    
    df = load_and_preprocess_course_data(course_data_file)
    doc_vectors = vectorize_text(df)
    
    process_test_cases_and_save(test_cases_file, course_data_file, output_file, doc_vectors)

/tmp/ipykernel_414/2973497617.py:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
Vectorizing documents: 100%|██████████| 4988/4988 [08:36<00:00,  9.67it/s]
/tmp/ipykernel_414/2973497617.py:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
Processing test cases: 100%|██████████| 50/50 [00:09<00:00,  5.06it/s]


Course recommendations saved to 'course_recommendations_bert.xlsx'
